A ideia desse projeto é usar fine tunning de parametros com o grod search, por exemplo

# Carregando a base de dados

In [1]:
import pandas as pd

X = pd.read_csv("../../bases/bases/entradas_breast.csv")
y = pd.read_csv("../../bases/bases/saidas_breast.csv")

In [2]:
import numpy as np

X = np.array(X, dtype="float32")
y = np.array(y, dtype="float32").squeeze(1)

# Classe para a rede neural

In [3]:
import torch.nn as nn
import torch.nn.functional as F
class classificador_torch(nn.Module):
    def __init__(
            self, 
            activation, 
            neurons, 
            initializer
        ):
        super().__init__() # pegando todos os atributos da super classe (nn.Module)
        
        self.dense0 = nn.Linear(in_features=30, out_features=neurons)
        initializer(self.dense0.weight) # inicializando a primeira camada com pesos uniformemente aleatorio
        self.activation0 = activation
        self.dropout0 = nn.Dropout(0.2)
        self.dense1 = nn.Linear(in_features=neurons, out_features=neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dropout1 = nn.Dropout(0.2)
        self.dense2 = nn.Linear(in_features=neurons, out_features=1)
        initializer(self.dense2.weight)
        self.output = nn.Sigmoid()
    def forward(self, x):
        x = self.dense0(x)
        x = self.activation0(x)
        x = self.dropout0(x)
        x = self.dense1(x)
        x = self.activation1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.output(x)
        return x

# Usando o SKORCH

In [4]:
from skorch import NeuralNetBinaryClassifier
import torch 

classificador_sk = NeuralNetBinaryClassifier(
    module=classificador_torch,
    lr=0.001,
    optimizer__weight_decay=0.0001,
    train_split=False
)

# Tunning de parâmetros

In [7]:
params = {
    "batch_size": [10, 30],
    "max_epochs": [50, 100],
    "optimizer": [torch.optim.Adam], # torch.optim.SGD],
    "criterion": [torch.nn.BCELoss], # torch.nn.HingeEmbeddingLoss],
    "module__activation": [F.relu], # F.tanh],
    "module__neurons": [8, 16],
    "module__initializer": [torch.nn.init.uniform_]# torch.nn.init.normal_]
}

In [10]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=classificador_sk,
    param_grid=params,
    scoring="accuracy",
    cv=2
)

grid_search = grid_search.fit(X, y)

  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.0398
      2       37.3239  0.0329
      3       37.3239  0.0346
      4       37.3239  0.0357
      5       37.3239  0.0359
      6       37.3239  0.0315
      7       37.3239  0.0347
      8       37.3239  0.0391
      9       37.3239  0.0366
     10       37.3239  0.0357
     11       37.3239  0.0345
     12       37.3239  0.0359
     13       37.3239  0.0366
     14       37.3239  0.0322
     15       37.3239  0.0361
     16       37.3239  0.0331
     17       37.3239  0.0332
     18       37.3239  0.0322
     19       37.3239  0.0359
     20       37.3239  0.0355
     21       37.3239  0.0421
     22       37.3239  0.0328
     23       37.3239  0.0369
     24       37.3239  0.0373
     25       37.3239  0.0338
     26       37.3239  0.0383
     27       37.3239  0.0444
     28       37.3239  0.0467
     29       37.3239  0.0336
     30       37.3240  0.0340
     31       37.3239  0.0404
     32   

In [12]:
grid_search.best_params_, grid_search.best_score_

({'batch_size': 10,
  'criterion': torch.nn.modules.loss.BCELoss,
  'max_epochs': 100,
  'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
  'module__neurons': 16,
  'optimizer': torch.optim.adam.Adam},
 np.float64(0.8576599950580677))